In [1]:
from utility.exps import run_real_experiments
#from utility.graphing_tools import generate_latex_table
import pandas as pd
import numpy as np

In [2]:
stock = run_real_experiments("stock", 200, cal_size=0.05, test_size=0.2)
stock.to_csv("real_exps/stock.csv")

e:\multi-target-scaling\utility\data_splitting.py:71: RuntimeWarning: invalid value encountered in divide
  scale = base_upper[reference_dim]/base_upper
e:\multi-target-scaling\utility\data_splitting.py:78: RuntimeWarning: invalid value encountered in divide
  adjustments = adj1*base_upper/base_upper[reference_dim]
e:\multi-target-scaling\utility\data_splitting.py:71: RuntimeWarning: invalid value encountered in divide
  scale = base_upper[reference_dim]/base_upper
e:\multi-target-scaling\utility\data_splitting.py:78: RuntimeWarning: invalid value encountered in divide
  adjustments = adj1*base_upper/base_upper[reference_dim]
e:\multi-target-scaling\utility\data_splitting.py:71: RuntimeWarning: invalid value encountered in divide
  scale = base_upper[reference_dim]/base_upper
e:\multi-target-scaling\utility\data_splitting.py:78: RuntimeWarning: invalid value encountered in divide
  adjustments = adj1*base_upper/base_upper[reference_dim]
e:\multi-target-scaling\utility\data_splitting.py

In [3]:
rf2 = run_real_experiments("rf2", 200, cal_size=0.05, test_size=0.2)
rf2.to_csv("real_exps/rf2.csv")

In [4]:
scm1d = run_real_experiments("scm1d", 200, cal_size=0.05, test_size=0.2)
scm1d.to_csv("real_exps/scm1d.csv")

In [5]:
scm20d = run_real_experiments("scm20d", 200, cal_size=0.05, test_size=0.2)
scm20d.to_csv("real_exps/scm20d.csv")

In [6]:
student_results = run_real_experiments("student", 200, cal_size=0.05, test_size=0.2)
student_results.to_csv("real_exps/student.csv")

In [7]:
energy_results = run_real_experiments("energy", 200, cal_size=0.05, test_size=0.2)
energy_results.to_csv("real_exps/energy.csv")

In [8]:
air_results = run_real_experiments("air", 200, cal_size=0.05, test_size=0.2)
air_results.to_csv("real_exps/air.csv")

In [9]:
crime_results = run_real_experiments("crime", 200, cal_size=0.05, test_size=0.2)
crime_results.to_csv("real_exps/crime.csv")

d:\anaconda\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [26]:
import pandas as pd
from textwrap import dedent

# ---------- Configuration ----------
datasets_panel1 = [
    ("Stock", "real_exps/stock.csv", 6, 15),
    ("rf2", "real_exps/rf2.csv", 8, 384),
]
datasets_panel2 = [
    ("scm1d", "real_exps/scm1d.csv", 16, 490),
    ("scm20d", "real_exps/scm20d.csv", 16, 448),
]
datasets_panel3 = [
    ("Energy", "real_exps/energy.csv", 2, 38),
    ("Student", "real_exps/student.csv", 3, 32),
]

methods_order = [
    "Bonferroni",
    "Unscaled",
    "Point CHR",
    "Empirical copula",
    "Standardized (Data Splitting)",
    "Standardized (Shortcut)"
]

# ---------- Helper formatting ----------
def format_pm(mean, std, sci=False, highlight=False):
    """Format as mean(std), optionally in scientific notation and highlighted."""
    if pd.isna(mean) or pd.isna(std):
        return f"\\text{{inf}}"
    if sci:
        mean_str = f"{mean:.3e}"
        std_str = f"{std:.3e}"
    else:
        mean_str = f"{mean:.3f}"
        std_str = f"{std:.3f}"
    result = f"{mean_str}({std_str})"
    if highlight:
        return f"\\textcolor{{red}}{{{result}}}"
    return result


def extract_stats(df):
    """Return dict[method] = (cov, cov_std, vol, vol_std)."""
    stats = {}
    for _, row in df.iterrows():
        m = row["Methods"].strip()
        stats[m] = (
            row["test_coverage_avg"],
            row["test_coverage_1std"],
            row["coverage_vol"],
            row["coverage_vol_1std"],
        )
    return stats

def build_panel(panel_datasets, caption):
    lines = []
    header_cols = " & ".join(
        [f"\\multicolumn{{2}}{{c}}{{{name} $(d={d},\\, n={n})$}}" for name, _, d, n in panel_datasets]
    )
    subheaders = " & ".join(["Coverage & Volume"] * len(panel_datasets))

    lines.append("\\begin{subtable}{\\textwidth}")
    lines.append("\\centering")
    lines.append("\\begin{tabular}{l" + "c c " * len(panel_datasets) + "}")
    lines.append("\\toprule")
    lines.append("\\textbf{Method} & " + header_cols + " \\\\")
    cmid = "".join([f"\\cmidrule(lr){{{2*i+2}-{2*i+3}}}" for i in range(len(panel_datasets))])
    lines.append(cmid)
    lines.append("& " + subheaders + " \\\\")
    lines.append("\\midrule")

    dfs = []
    for _, file, _, _ in panel_datasets:
        df = pd.read_csv(file)
        dfs.append(extract_stats(df))

    # Find min volume per dataset for highlighting
    min_vols = []
    for data in dfs:
        vols = [v[2] for v in data.values() if pd.notna(v[2])]
        min_vols.append(min(vols) if vols else None)

    # Build each row (method)
    for method in methods_order:
        row_parts = [method]
        for j, data in enumerate(dfs):
            if method in data:
                cov, cov_std, vol, vol_std = data[method]
                cov_str = format_pm(cov, cov_std, sci=False)
                highlight = (vol == min_vols[j])
                vol_str = format_pm(vol, vol_std, sci=True, highlight=False)
            else:
                cov_str, vol_str = "--", "--"
            row_parts += [cov_str, vol_str]
        lines.append(" & ".join(row_parts) + " \\\\")

    lines.append("\\bottomrule")
    lines.append("\\end{tabular}")
    lines.append(f"\\subcaption{{{caption}}}")
    lines.append("\\end{subtable}")
    return "\n".join(lines)



# ---------- Build full table ----------
panel_a = build_panel(datasets_panel1, "First panel: Stock, rf2.")
panel_b = build_panel(datasets_panel2, "Second panel: scm1d, scm20d.")
panel_c = build_panel(datasets_panel3, "Third panel: Energy, Student.")

latex_table = dedent(f"""
\\begin{{table}}[!htbp]
\\centering
\\captionsetup[subtable]{{justification=centering}}
\\small
{panel_a}

\\vspace{{0.35em}}

{panel_b}

\\vspace{{0.35em}}

{panel_c}


\\caption{{Metrics on real data. We report mean(std) over seeds. 
Target coverage is $0.9$. For efficiency (volume), lower is better.
The lowest (best) efficiency per dataset is highlighted in red.}}
\\label{{tab:real}}
\\end{{table}}
""")

with open("realdata_table_highlighted.tex", "w") as f:
    f.write(latex_table)

print("✅ LaTeX table with red highlights saved to realdata_table_highlighted.tex")

✅ LaTeX table with red highlights saved to realdata_table_highlighted.tex
